# RDF2MARCXML

Replace the URI with your Instance URI from Sinopia to generate MARCXML based on the Library of Congress BIBFRAME2MARC mapping.


In [1]:
!pip install subprocess.run

In [2]:
import subprocess
from marc_xml.lc_bfxml_work import lc_bfxml_work, remove_last_line
from marc_xml.lc_bfxml_instance import lc_bfxml_instance, remove_rdf_header

#replace uri with the Instance uri of the resource you want to convert
uri = "https://api.development.sinopia.io/resource/90fb418e-4ff4-4296-af69-1e30e2ac2f8a"

lc_bfxml_work(uri)  
remove_last_line() 
lc_bfxml_instance(uri) 
remove_rdf_header() 


In [3]:
# combine the two files use work first then add the instance
with open("lc_bfxml_work.xml", "r") as work_file:
    work = work_file.read()
with open("lc_bfxml_instance.xml", "r") as instance_file:
    instance = instance_file.read()

#save as a file name LoC_Work_Instance.xml
with open("LoC_Work_Instance.xml", "w") as combined_file:
    combined_file.write(work + instance)

# add the sinopiabf namespace to the combined file
with open("LoC_Work_Instance.xml", "r") as file:
    filedata = file.read()
filedata = filedata.replace('<rdf:RDF', '<rdf:RDF xmlns:sinopiabf="http://sinopia.io/vocabulary/bf/"')
with open("LoC_Work_Instance.xml", "w") as file:
    file.write(filedata)

# apply "pre-transform-normalize.xsl" on LoC_Work_Instance.xml file for normalization
import lxml.etree as ET
dom = ET.parse("LoC_Work_Instance.xml")
xslt = ET.parse("marc_xml/xsl/pre-transform-normalize.xsl")
transform = ET.XSLT(xslt)
newdom = transform(dom)
with open("LoC_Work_Instance_Normalized.xml", "w") as f:
    f.write(str(newdom))    


# use xsltproc for bibfame to mark conversion
subprocess.run(['xsltproc', '-o', 'marc.xml', '../bibframe2marc/bibframe2marc.xsl', 'LoC_Work_Instance_Normalized.xml'])

Record idp140160675494879: Unprocessed node bf:language. Non-repeatable target element 008.
Record idp140160675494879: Unprocessed node bf:assigner. Non-repeatable target field (040).


CompletedProcess(args=['xsltproc', '-o', 'marc.xml', '../bibframe2marc/bibframe2marc.xsl', 'LoC_Work_Instance_Normalized.xml'], returncode=0)